# Togo Solar Data EDA

**Objective:** Profile, clean, and explore Togo’s solar dataset to prepare it for comparison and region-ranking tasks. This notebook covers summary statistics, missing value analysis, outlier detection, cleaning, and exploratory visualizations.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8')

In [ ]:
# Load the Togo solar dataset
file_path = '../data/togo-dapaong_qc.csv'
df = pd.read_csv(file_path, parse_dates=['Timestamp'], infer_datetime_format=True)
df.head()

In [ ]:
# Summary statistics for numeric columns
summary_stats = df.describe()
display(summary_stats)

# Missing value report
missing_report = df.isna().sum().to_frame('missing_count')
missing_report['missing_pct'] = 100 * missing_report['missing_count'] / len(df)
missing_report = missing_report[missing_report['missing_count'] > 0]
display(missing_report)

# List columns with >5% nulls
high_null_cols = missing_report[missing_report['missing_pct'] > 5]
high_null_cols

In [ ]:
# Outlier detection using Z-score for key columns
key_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
for col in key_cols:
    if col in df.columns:
        df[col + '_zscore'] = zscore(df[col].astype(float), nan_policy='omit')
        df[col + '_outlier'] = df[col + '_zscore'].abs() > 3

# Count outliers per column
outlier_counts = {col: df[col + '_outlier'].sum() for col in key_cols if col + '_outlier' in df.columns}
outlier_counts

In [ ]:
# Impute or drop missing values in key columns
for col in key_cols:
    if col in df.columns:
        if df[col].isna().sum() > 0:
            median_val = df[col].median()
            df[col].fillna(median_val, inplace=True)

df.dropna(subset=key_cols, inplace=True)

# Remove outliers
for col in key_cols:
    if col + '_outlier' in df.columns:
        df = df[~df[col + '_outlier']]

df.drop(columns=[c for c in df.columns if c.endswith('_zscore') or c.endswith('_outlier')], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

In [ ]:
# Export cleaned DataFrame to CSV (do not commit CSVs to git)
clean_path = '../data/togo_clean.csv'
df.to_csv(clean_path, index=False)
print(f'Cleaned data exported to {clean_path}')

In [ ]:
# Time Series Analysis: GHI, DNI, DHI, Tamb vs. Timestamp
plot_cols = ['GHI', 'DNI', 'DHI', 'Tamb']
fig, axs = plt.subplots(len(plot_cols), 1, figsize=(14, 3 * len(plot_cols)), sharex=True)
for i, col in enumerate(plot_cols):
    if col in df.columns:
        axs[i].plot(df['Timestamp'], df[col], label=col)
        axs[i].set_ylabel(col)
        axs[i].legend()
axs[-1].set_xlabel('Timestamp')
plt.tight_layout()
plt.show()

In [ ]:
# Cleaning Impact: Average ModA & ModB pre/post-clean
if 'Cleaning' in df.columns:
    cleaning_group = df.groupby('Cleaning')[['ModA', 'ModB']].mean().reset_index()
    cleaning_group.plot(x='Cleaning', y=['ModA', 'ModB'], kind='bar', figsize=(8,5))
    plt.title('Average ModA & ModB Pre/Post Clean')
    plt.ylabel('Average Value')
    plt.show()
else:
    print('No Cleaning flag column found in data.')

In [ ]:
# Correlation Heatmap
corr_cols = ['GHI', 'DNI', 'DHI', 'TModA', 'TModB']
corr_data = df[[col for col in corr_cols if col in df.columns]]
plt.figure(figsize=(8,6))
sns.heatmap(corr_data.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap')
plt.show()

# Scatter plots: WS, WSgust, WD vs. GHI; RH vs. Tamb or RH vs. GHI
scatter_pairs = [
    ('WS', 'GHI'),
    ('WSgust', 'GHI'),
    ('WD', 'GHI'),
    ('RH', 'Tamb'),
    ('RH', 'GHI')
]
for x, y in scatter_pairs:
    if x in df.columns and y in df.columns:
        plt.figure(figsize=(6,4))
        sns.scatterplot(data=df, x=x, y=y, alpha=0.5)
        plt.title(f'{x} vs. {y}')
        plt.show()

In [ ]:
# Wind Rose / Radial Bar Plot for WS/WD
if 'WS' in df.columns and 'WD' in df.columns:
    import plotly.figure_factory as ff
    fig = px.bar_polar(df, r='WS', theta='WD', color='WS',
                      color_continuous_scale=px.colors.sequential.Plasma,
                      title='Wind Rose: Wind Speed by Direction')
    fig.show()
else:
    print('WS or WD column not found for wind rose plot.')

# Histograms for GHI and WS
for col in ['GHI', 'WS']:
    if col in df.columns:
        plt.figure(figsize=(6,4))
        sns.histplot(df[col], bins=30, kde=True)
        plt.title(f'Histogram of {col}')
        plt.xlabel(col)
        plt.ylabel('Frequency')
        plt.show()

In [ ]:
# Temperature Analysis: RH influence on Tamb and GHI
if 'RH' in df.columns and 'Tamb' in df.columns:
    plt.figure(figsize=(6,4))
    sns.scatterplot(data=df, x='RH', y='Tamb', alpha=0.5)
    plt.title('Relative Humidity vs. Ambient Temperature')
    plt.show()
if 'RH' in df.columns and 'GHI' in df.columns:
    plt.figure(figsize=(6,4))
    sns.scatterplot(data=df, x='RH', y='GHI', alpha=0.5)
    plt.title('Relative Humidity vs. GHI')
    plt.show()

# Bubble Chart: GHI vs. Tamb, bubble size = RH or BP
bubble_size = 'RH' if 'RH' in df.columns else ('BP' if 'BP' in df.columns else None)
if 'GHI' in df.columns and 'Tamb' in df.columns and bubble_size:
    fig = px.scatter(df, x='GHI', y='Tamb', size=bubble_size, color=bubble_size,
                     title=f'GHI vs. Tamb (Bubble size = {bubble_size})',
                     labels={'GHI': 'Global Horizontal Irradiance', 'Tamb': 'Ambient Temperature', bubble_size: bubble_size})
    fig.show()
else:
    print('Required columns for bubble chart not found.')

## EDA Best Practices & References
- Always inspect data types, missing values, and outliers before analysis.
- Use visualizations (line, bar, scatter, heatmap, histograms) to uncover trends and relationships.
- Impute or drop missing/outlier values based on domain knowledge and data distribution.
- Document cleaning steps for reproducibility.
- For time series, check for seasonality, trends, and anomalies.
- Use correlation and scatter plots to identify key drivers and relationships.
- For wind data, wind rose plots are effective for directionality.
- Share actionable insights, not just plots.

**References:**
- [Pandas EDA Guide](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html)
- [Seaborn Statistical Data Visualization](https://seaborn.pydata.org/examples/index.html)
- [Plotly Express Gallery](https://plotly.com/python/plotly-express/)
- [Wind Rose Plotting in Python](https://python-windrose.readthedocs.io/en/latest/)
- [Data Cleaning Techniques](https://www.dataquest.io/blog/data-cleaning/)

*This notebook demonstrates a full EDA workflow, preparing the Togo solar dataset for further analysis and comparison.*